In [1]:
#gurobi optimizer
import gurobipy as gp
from gurobipy import GRB

import numpy as np

import pandas as pd

In [2]:
model = gp.Model("VRPLTT")

Restricted license - for non-production use only - expires 2023-10-25


In [3]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [4]:
load_levels

array([ 20.,  60., 100., 140.])

In [5]:
upper

array([ 40.,  80., 120., 160.])

In [6]:

def vel(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eff = 0.95
    
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))/eff
    c3 = c_Fd/eff
    
    coefs = [c3, 0, c1, -P]
    #print(f"{np.real(np.roots(coefs))=}")
    v = np.max(np.real(3.6*np.roots(coefs)))
    return min(v,30)
    #return 3.6*np.roots(coefs)
    

def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P)
    return np.round(t_ij_mat,4)

The forces acting on the bicyle are: the aerodynamic drag resistance $F_{D}$, the rolling resistance $F_{R}$ and the component of the gravity force along the direction of motion $F_{G}$:

* $ F_{D} = \frac{\rho C_{D}Av^{2}}{2}$
* $ F_{R} = C_{R}mg \cdot cos(arctan(h))$
* $ F_{G} = mg \cdot sin(arctan(h))$

We have to distinguish two cases: uphill and downhill.
In uphill, the component of the gravity force along the direction of motion is a resistance force, while in downhill it helps the motion of the bicyle.
so we have, at equilibrium:

* uphill (1): $\mathrm{ \eta P_{traction} = (F_{D} + (F_{R} + F_{G})v }$
* downhill (2): $\mathrm{ \eta P_{traction} + F_{G}v = (F_{D} + F_{R})v }$

where $\eta$ is the efficiency of the mechanic transmission, which is assumed to be 0.95.

Moreover, we assume that the bicyle speed is limited in downhill for safety reasons. Within cities, a typical speed limit for for motor vehicles is 50km/h. Considering also that the regulation for pedelec bicyle allows a maximum speed of 25km/h in electric assisted mode, we set a maximum speed in downhill of 30km/h.
The delta energy can be used for recharging the battery for example, similarly to other solutions in the automaotive industry.
The maximum speed limit is imposed on uphill as well.
The strategy for solving the equation is:
* uphill: solve equation (1) and cap the velocity to 25 km/h. We assume that in uphill, the cyclist always use electric power assistance.
* downhill: solve equation (2) with $\overline{P}=$ cyclist power, for ex. $\overline{P}=$ 150 Watt. If $v_{solution} \geq $ 30km/h set $v=30km/h$, while if $25 km/h \leq v_{solution} \leq 30 km/h$ set $v = v_{solution}$, otherwise reset the equation by using $\overline{P} = P_{cyclist} + 250$ and solve again for the velocity $v_{solution}$: use the calculated value if $v_{solution} \leq 25 km/h$, otherwise set $v=25 km/h$.


The equation for $v$ at constant speed can be reduced to the form $y^{3} + py + q = 0$, for which the following formula holds for the roots:

$y = \sqrt[3]{-\frac{q}{2} + \sqrt{\frac{q^2}{4} + \frac{p^{3}}{27}} } + \sqrt[3]{-\frac{q}{2} - \sqrt{\frac{q^2}{4} + \frac{p^{3}}{27}} }$

If we use a negative h in the formuals above when traversing an edge with negative slope, since $cos(\alpha) = cos(-\alpha)$ and $sin(\alpha) = -sin(-\alpha)$, we have just one expression for the coefficient p:

$ p = \frac{mg[C_{R}cos(arctan(h) + sin(arctan(h)]}{\rho C_{D}Av^{2}} $

while, for q:
* uphill: 

$ q = -\frac{\eta (\overline{P_{cyclist}}+P_{electric})}{\rho C_{D}Av^{2}} $

* downhill: 

$ q_{max\_speed} = -\frac{\eta \overline{P_{cyclist}}}{\rho C_{D}Av^{2}} \mathrm \ $ if $25 km/h \leq v_{solution} \leq 30km/h \mathrm \ $, with the limit $v = 30 km/h$ if $v_{solution} \gt 30 km/h$ or

$ q_{assisted} = -\frac{\eta (\overline{P_{cyclist}}+P_{electric})}{\rho C_{D}Av^{2}} \mathrm \ $ if $v_{solution} \leq 25km/h$, with the limit $v = 25 km/h$ if $v_{new\_solution} \gt 25 km/h$

In practice we have $P_{electric}$ = 250W, while for $\overline {P_{cyclist}}$ we assume to have few different types of cyclist.

In [1]:
def vel1(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eta = 0.95
    
    c0 = P*eta
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))
    c3 = c_Fd
    
   ### closed form solution of 3-rd degree polynomial in the form
   ### ....

In [24]:
df = pd.read_csv("./instances/large/fukuoka_full.csv")
P = 500 # Watt

t=time_matrix(df,load_levels,P=P)

In [25]:
from pprint import pprint
print(t[3,4])
print(t[4,3])


[2.556 2.556 2.556 2.556]
[ 6.4225 18.4472 30.7002 42.9692]


In [26]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def W(u,v,weight_matrix):
    for i in adj[u]:
        if i.end==v:
            return i.weight
        
def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 13.8463,
 2: 12.3422,
 3: 12.280999999999999,
 4: 13.049299999999999,
 5: 8.1761,
 6: 10.2227,
 7: 12.6424,
 8: 5.4474,
 9: 13.209200000000001,
 10: 12.7395,
 11: 9.5632,
 12: 8.1479,
 13: 9.1759,
 14: 6.933,
 15: 9.2201,
 16: 12.9502,
 17: 11.980599999999999,
 18: 9.3014,
 19: 7.1181,
 20: 8.9259,
 21: 11.5398,
 22: 8.7414,
 23: 11.2575,
 24: 11.4766,
 25: 9.9359,
 26: 11.245499999999998,
 27: 12.3935,
 28: 7.3896999999999995,
 29: 11.0881,
 30: 11.9574,
 31: 9.9584,
 32: 12.4942,
 33: 10.811499999999999,
 34: 12.5434,
 35: 11.3924,
 36: 10.909399999999998,
 37: 9.0481,
 38: 10.5912,
 39: 12.3261,
 40: 13.058900000000001,
 41: 8.6924,
 42: 13.070900000000002,
 43: 11.4564,
 44: 9.094199999999999,
 45: 9.4701,
 46: 7.9142,
 47: 8.444199999999999,
 48: 11.570400000000001,
 49: 13.945699999999999,
 50: 7.8677,
 51: 10.4613,
 52: 6.715,
 53: 6.3172,
 54: 12.7741,
 55: 9.2357,
 56: 11.846499999999999,
 57: 8.6482,
 58: 12.7834,
 59: 11.767199999999999,
 60: 12.8541,
 61: 8.8981,

In [27]:
for i in t[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	20.447 	19.079 	16.716 	17.061 	8.176 	10.302 	13.616 	5.447 	20.998 	17.52 	10.101 	9.608 	11.96 	7.296 	11.905 	21.086 	16.026 	11.133 	9.033 	10.167 	15.471 	8.743 	11.618 	17.41 	14.345 	16.162 	16.393 	8.983 	14.064 	18.701 	13.517 	19.433 	14.928 	18.088 	15.461 	15.062 	11.292 	13.507 	18.918 	15.12 	10.44 	22.02 	15.622 	11.632 	12.56 	7.914 	9.811 	16.017 	18.032 	7.868 	10.509 	7.73 	6.372 	13.975 	9.334 	13.386 	10.379 	17.538 	16.63 	15.212 	10.966 	16.335 	9.955 	10.891 	14.692 	12.752 	15.167 	11.091 	14.157 	16.521 	12.411 	17.321 	9.647 	10.374 	7.606 	16.314 	8.15 	10.147 	15.628 	17.442 	12.139 	11.406 	14.24 	13.445 	15.312 	12.93 	11.288 	12.913 	10.629 	10.419 	18.619 	8.529 	13.588 	13.398 	20.956 	16.789 	15.473 	18.328 	10.19 	18.67 

	5.924 	0.0 	0.43 	1.332 	3.648 	6.104 	6.692 	7.65 	3.884 	0.27 	3.806 	6.66 	3.352 	2.164 	3.51 	2.26 	0.344 	1.588 	4.7 	4.362 	5.398 	1.662 	6.268 	7.064 	1.564 	3.064 	3.606 	3.46 	3.232 	2.794 	1.444 	1.868 	0.39 	3.172

In [28]:
%%timeit
dist,parent = dijkstra(0,t)


5.76 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    print(levels)
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix



In [31]:
sp = shortest_path_matrix(t)
for i in sp[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")
    

[0 1 2 3]
	0.0 	13.846 	12.342 	12.281 	13.049 	8.176 	10.223 	12.642 	5.447 	13.209 	12.74 	9.563 	8.148 	9.176 	6.933 	9.22 	12.95 	11.981 	9.301 	7.118 	8.926 	11.54 	8.741 	11.258 	11.477 	9.936 	11.245 	12.394 	7.39 	11.088 	11.957 	9.958 	12.494 	10.811 	12.543 	11.392 	10.909 	9.048 	10.591 	12.326 	13.059 	8.692 	13.071 	11.456 	9.094 	9.47 	7.914 	8.444 	11.57 	13.946 	7.868 	10.461 	6.715 	6.317 	12.774 	9.236 	11.846 	8.648 	12.783 	11.767 	12.854 	8.898 	12.597 	8.593 	8.93 	10.763 	10.608 	11.204 	10.986 	9.903 	11.976 	9.334 	12.673 	8.39 	8.63 	6.489 	11.292 	7.335 	8.09 	11.708 	12.72 	11.32 	11.014 	11.263 	10.199 	12.818 	10.061 	11.288 	9.5 	8.556 	10.335 	12.019 	6.94 	12.614 	10.438 	14.329 	12.493 	12.87 	13.021 	9.451 	12.888 

	5.414 	0.0 	0.43 	1.332 	3.648 	4.574 	4.172 	3.826 	3.374 	0.27 	3.71 	4.388 	3.024 	2.16 	3.18 	2.256 	0.344 	1.588 	3.364 	3.74 	4.812 	1.658 	4.546 	3.966 	1.562 	3.06 	3.334 	3.458 	2.902 	2.79 	1.444 	1.866 	0.39 	3.168 	1.278 	2.92